<a href="https://colab.research.google.com/github/gururaja-ai/ChatUPT.github.io/blob/master/LSTM%2BGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries & Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
# Read the dataset
df = pd.read_csv('/content/drive/MyDrive/Deep_Learning_Project/train.csv')

# Display the dataset
print(df.head())


In [ ]:
# Preprocessing and feature engineering
pcu_values = {
    "CAR": 1,
    "LCV": 1.5,
    "BUS": 3,
    "TRUCK": 3,
    "MAV": 4.5
}

df['PCU'] = df['CAR'] * pcu_values['CAR'] + \
            df['LCV'] * pcu_values['LCV'] + \
            df['BUS'] * pcu_values['BUS'] + \
            df['TRUCK'] * pcu_values['TRUCK'] + \
            df['MAV'] * pcu_values['MAV']

df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Calculate the number of days in each month
days_in_month = df['Date'].dt.days_in_month
# Calculate average daily PCU (ADT)
df['ADT'] = df['PCU'] / days_in_month
df['Date'] = df['Date'].dt.strftime('%Y-%m')

numeric_columns = ['CAR', 'LCV', 'BUS', 'TRUCK', 'MAV','PCU']
# Convert numeric columns to integers
df[numeric_columns + ['ADT']] = df[numeric_columns + ['ADT']].astype(int)

# Display the preprocessed dataset
print(df.head())


In [ ]:
# Plot the date and ADT values in a line chart
plt.figure(figsize=(16, 8))
plt.plot(df['Date'], df['ADT'], color='blue')
plt.xlabel('Date')
plt.ylabel('ADT')
plt.title('Average Daily Traffic (in PCU) over Time')
plt.show()


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Define the number of splits (here, we'll use 2 for training and testing)
n_splits = 2

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=n_splits)

# Initialize lists to store train and test data
train_data_list = []
test_data_list = []

# Split the data using TimeSeriesSplit
for train_index, test_index in tscv.split(df):
    train_data_list.append(df.iloc[train_index])
    test_data_list.append(df.iloc[test_index])

# Concatenate the train and test data from all splits
train_data = pd.concat(train_data_list)
test_data = pd.concat(test_data_list)

# Save the training and testing data to CSV files
train_data[['Date', 'ADT']].to_csv('train_data.csv', index=False)
test_data[['Date', 'ADT']].to_csv('test_data.csv', index=False)

# Display the shape of the datasets
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)


In [ ]:
# Get the last date in the test data
last_date = test_data['Date'].max()

# Convert the 'Date' column to datetime format
test_data['Date'] = pd.to_datetime(test_data['Date'])

# Calculate the start date for forecasting by adding one month to the last date
start_date = pd.to_datetime(last_date) + pd.DateOffset(months=1)

# Define the number of forecasted periods
num_periods = 36

# Generate forecasted months
forecast_months = pd.date_range(start=start_date, periods=num_periods, freq='MS')

# Create DataFrame with forecasted months
forecast_df = pd.DataFrame({'Date': forecast_months.strftime('%Y-%m'), 'ADT': [None] * num_periods})

# Save the forecast data to a CSV file
forecast_df.to_csv('forecasted_months.csv', index=False)

# Display the forecasted dataframe
print(forecast_df.head())

In [ ]:
# Scale the data
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data[['ADT']])
test_data_scaled = scaler.transform(test_data[['ADT']])

# Display the scaled training data
print(train_data_scaled[:5])


In [ ]:
# Create a windowed dataset for the LSTM model
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:(i + window_size)])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

window_size = 12
X_train, y_train = create_dataset(train_data_scaled, window_size)
X_test, y_test = create_dataset(test_data_scaled, window_size)

# Reshape input to be 3D [samples, timesteps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Display the shape of the training and testing data
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
from tensorflow.keras.optimizers import Adam

# Create the LSTM model with modified parameters
model = Sequential()
model.add(LSTM(units=200, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=200))
model.add(Dense(units=1))

# Compile the model with custom learning rate using Adam optimizer
custom_optimizer = Adam(lr=0.0001)  # Setting learning rate to 0.0001
model.compile(optimizer=custom_optimizer, loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), verbose=1)


In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# Forecasting ADT for the forecasted months
forecast_data_scaled = scaler.transform(forecast_df[['ADT']])
forecast_data = []

# Seed the last window of training data into the model to forecast future ADT
current_batch = train_data_scaled[-window_size:].reshape((1, window_size, 1))

for i in range(num_periods):
    # Predict the next value
    forecast = model.predict(current_batch)[0]

    # Append the forecasted value to the forecast data
    forecast_data.append(forecast)

    # Update the current batch to include the forecasted value
    current_batch = np.append(current_batch[:,1:,:],[[forecast]],axis=1)

# Inverse scaling on forecasted data
forecast_data = scaler.inverse_transform(forecast_data)

# Update the forecasted ADT in the forecast dataframe
forecast_df['ADT'] = forecast_data

# Save the updated forecast data to a CSV file
forecast_df.to_csv('forecasted_months.csv', index=False)

# Display the forecasted dataframe
print(forecast_df.head())


In [ ]:
from tensorflow.keras.layers import GRU

# Create the GRU model
model_gru = Sequential()
model_gru.add(GRU(units=200, input_shape=(X_train.shape[1], 1)))
model_gru.add(Dense(units=5))

# Compile the GRU model
model_gru.compile(optimizer='adam', loss='mean_squared_error')

# Train the GRU model
history_gru = model_gru.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), verbose=1)


In [ ]:
# Forecasting ADT for the forecasted months using GRU
forecast_data_gru = []

# Seed the last window of training data into the GRU model to forecast future ADT
current_batch_gru = train_data_scaled[-window_size:].reshape((1, window_size, 1))

for i in range(num_periods):
    # Predict the next value using GRU model
    forecast_gru = model_gru.predict(current_batch_gru)[0]

    # Append the forecasted value to the forecast data
    forecast_data_gru.append(forecast_gru)

    # Update the current batch to include the forecasted value
    current_batch_gru = np.append(current_batch_gru[:,1:,:],[[forecast_gru]],axis=1)

# Inverse scaling on forecasted data using GRU model
forecast_data_gru = scaler.inverse_transform(forecast_data_gru)

# Update the forecasted ADT in the forecast dataframe for GRU
forecast_df['ADT_GRU'] = forecast_data_gru

# Save the updated forecast data to a new CSV file
forecast_df.to_csv('forecasted_months_with_gru.csv', index=False)

# Display the forecasted dataframe with GRU
print(forecast_df.head())


In [ ]:
# Forecast using LSTM
forecast_data_lstm = []

# Seed the last window of training data into the LSTM model to forecast future ADT
current_batch_lstm = train_data_scaled[-window_size:].reshape((1, window_size, 1))

for i in range(num_periods):
    # Predict the next value using LSTM model
    forecast_lstm = model.predict(current_batch_lstm)[0]

    # Append the forecasted value to the forecast data
    forecast_data_lstm.append(forecast_lstm)

    # Update the current batch to include the forecasted value
    current_batch_lstm = np.append(current_batch_lstm[:,1:,:],[[forecast_lstm]],axis=1)

# Inverse scaling on forecasted data using LSTM model
forecast_data_lstm = scaler.inverse_transform(forecast_data_lstm)

# Update the forecasted ADT in the forecast dataframe for LSTM
forecast_df['ADT_LSTM'] = forecast_data_lstm

In [ ]:
# Ensure that num_periods matches the length of the forecasted data
num_periods = len(test_data)

# Calculate RMSE for LSTM and GRU forecasts
rmse_lstm = sqrt(mean_squared_error(test_data['ADT'].values, forecast_data_lstm[:num_periods]))
rmse_gru = sqrt(mean_squared_error(test_data['ADT'].values, forecast_data_gru[:num_periods]))

print("RMSE for LSTM Forecast:", rmse_lstm)
print("RMSE for GRU Forecast:", rmse_gru)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

In [ ]:
# prompt: other metrics to be evaluated

# Calculate Mean Absolute Error (MAE) for LSTM and GRU forecasts
mae_lstm = mean_absolute_error(test_data['ADT'].values, forecast_data_lstm[:num_periods])
mae_gru = mean_absolute_error(test_data['ADT'].values, forecast_data_gru[:num_periods])

# Calculate Mean Absolute Percentage Error (MAPE) for LSTM and GRU forecasts
mape_lstm = mean_absolute_percentage_error(test_data['ADT'].values, forecast_data_lstm[:num_periods])
mape_gru = mean_absolute_percentage_error(test_data['ADT'].values, forecast_data_gru[:num_periods])

# Calculate R2 score for LSTM and GRU forecasts
r2_lstm = r2_score(test_data['ADT'].values, forecast_data_lstm[:num_periods])
r2_gru = r2_score(test_data['ADT'].values, forecast_data_gru[:num_periods])

# Print the evaluation metrics
print("MAE for LSTM Forecast:", mae_lstm)
print("MAE for GRU Forecast:", mae_gru)
print("MAPE for LSTM Forecast:", mape_lstm)
print("MAPE for GRU Forecast:", mape_gru)
print("R2 Score for LSTM Forecast:", r2_lstm)
print("R2 Score for GRU Forecast:", r2_gru)


In [ ]:
# Plot the training and testing data along with forecasted values for LSTM and GRU
plt.figure(figsize=(16, 8))
plt.plot(train_data['ADT'], label='Training Data')
plt.plot(test_data['ADT'], label='Testing Data')
plt.plot(forecast_df['ADT_LSTM'], label='LSTM Forecast')
plt.plot(forecast_df['ADT_GRU'], label='GRU Forecast')
plt.xlabel('Date')
plt.ylabel('ADT')
plt.title('LSTM and GRU Forecasts vs. Actual Data')
plt.legend()
plt.show()

# Plot the forecasted values for LSTM and GRU
plt.figure(figsize=(16, 8))
plt.plot(forecast_df['ADT_LSTM'], label='LSTM Forecast')
plt.plot(forecast_df['ADT_GRU'], label='GRU Forecast')
plt.xlabel('Date')
plt.ylabel('ADT')
plt.title('LSTM and GRU Forecasts')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.optimizers import Adam

# Define a function to create and compile LSTM model with given hyperparameters
def create_lstm_model(units=100, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=units))
    model.add(Dense(units=1))

    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Perform hyperparameter tuning
best_rmse = float('inf')
best_params = {}

for units in [50, 100, 150]:
    for learning_rate in [0.001, 0.01, 0.1]:
        lstm_model = create_lstm_model(units=units, learning_rate=learning_rate)
        history = lstm_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=0)
        rmse = sqrt(history.history['val_loss'][-1])  # Get RMSE from validation loss
        if rmse < best_rmse:
            best_rmse = rmse
            best_params['units'] = units
            best_params['learning_rate'] = learning_rate

print("Best LSTM Hyperparameters:", best_params)


In [ ]:
# Define a function to create and compile LSTM model with stacked layers
def create_stacked_lstm_model(units=150, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=units, return_sequences=True))
    model.add(LSTM(units=units))
    model.add(Dense(units=1))

    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Train the stacked LSTM model
stacked_lstm_model = create_stacked_lstm_model(units=best_params['units'], learning_rate=best_params['learning_rate'])
history_stacked_lstm = stacked_lstm_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)



In [ ]:
# Convert 'Date' column to string format
test_data['Date'] = test_data['Date'].astype(str)

# Plot the training and testing data along with forecasted values for LSTM and GRU
plt.figure(figsize=(16, 8))
plt.plot(train_data['Date'], train_data['ADT'], label='Training Data')
plt.plot(test_data['Date'], test_data['ADT'], label='Testing Data')
plt.plot(forecast_df['Date'], forecast_df['ADT_LSTM'], label='LSTM Forecast')
plt.plot(forecast_df['Date'], forecast_df['ADT_GRU'], label='GRU Forecast')
plt.xlabel('Date')
plt.ylabel('ADT')
plt.title('LSTM and GRU Forecasts vs. Actual Data')
plt.legend()
plt.show()


# Conclusion:

Despite multiple iterations and hyperparameter tuning, Model results are not satisfactory as